<a href="https://colab.research.google.com/github/bachvu98/Policy-NLP/blob/master/Preprocessing_Policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import required dependencies
First, we import the require dependencies

In [19]:
import pandas as pd
import numpy as np
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [12]:
segments = pd.read_csv('/content/drive/My Drive/OPP-115/OPP-115/segment_categories.csv',index_col=0)
segments.head()

,Policy UID,segment_id,category_name,segments
0,20,0,Other,<strong> Privacy Policy </strong> <br> <br> <s...
1,20,1,Other,This privacy policy does not apply to Sites ma...
2,20,2,Policy Change,"By visiting our Sites, you are accepting the p..."
3,20,3,First Party Collection/Use,<strong> What Information Is Collected? </stro...
4,20,4,First Party Collection/Use,<strong> Personally Identifiable Information <...


# Preprocessing segments text

In [15]:
def clean_text(text):
  text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text).split())
  #Then tokenisation
  tokens = word_tokenize(text)
  # convert to lower case
  tokens = [w.lower() for w in tokens]
  # remove punctuation from each word
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  # remove remaining tokens that are not alphabetic
  words = [word for word in stripped if word.isalpha()]
  # filter out stop words
  stop_words = set(stopwords.words('english'))
  # You can add more stop words here, specific for texts
  words = [w for w in words if not w in stop_words]
  # stemming of words
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(word) for word in words]
  # Convert from list to a sentence again
  text = ' '.join(word for word in words)
  return text

In [20]:
#Process the segments here
segments['segments'] = segments['segments'].apply(clean_text)
segments.head()

,Policy UID,segment_id,category_name,segments
0,20,0,Other,strong privacy policy strong br br strong effe...
1,20,1,Other,privacy policy apply site maintained company o...
2,20,2,Policy Change,visiting site accepting practice described pri...
3,20,3,First Party Collection/Use,strong information collected strong br br coll...
4,20,4,First Party Collection/Use,strong personally identifiable information str...


In [21]:
segments.iloc[5]['segments']

'strong nonpersonally identifiable information strong br br interact site may collect certain information identify individually nonpersonally identifiable information nonpii demographic data information computer mobile device device use access site including ip address general location information unique device identifier browser type browser language transactional information may also collect information use site including search term search result additional traffic data time access date access software crash report session identification number access time referring website address server may automatically keep activity log use site addition nonidentifiable individual information may collect aggregate data regarding use site br br'

In [22]:
segments.to_csv("/content/drive/My Drive/OPP-115/OPP-115/segment_processed.csv")